In [ ]:
import pandas as pd

# Load the pickle file back into a DataFrame
data = pd.read_pickle('data/simulated_data.pkl')



# Check the loaded data
print(data.head())

unique_id_count = data['id'].nunique()
print(f"Number of unique IDs: {unique_id_count}")

print(data.shape)

   id  visit  obstime  predtime  time  event         Y1        Y2        Y3  \
0   0      0        0         0     9   True  11.943728 -3.032593  2.760192   
1   0      1        1         1     9   True  12.255357 -5.431790  4.225383   
2   0      2        2         2     9   True  12.491947 -6.953460  2.854653   
3   0      3        3         3     9   True  16.406431 -8.508030  4.766191   
4   0      4        4         4     9   True  16.632347 -9.813989  5.816555   

    X1        X2    pred_Y1   pred_Y2   pred_Y3      true  
0  1.0  0.680195  11.943728 -3.032593  2.760192  1.000000  
1  1.0  0.680195  12.255357 -5.431790  4.225383  0.999397  
2  1.0  0.680195  12.491947 -6.953460  2.854653  0.998135  
3  1.0  0.680195  16.406431 -8.508030  4.766191  0.995494  
4  1.0  0.680195  16.632347 -9.813989  5.816555  0.989983  
Number of unique IDs: 1000


In [30]:
df = pd.read_pickle("data/simulated_data.pkl")      # ① 读取 pickle（通常是 DataFrame）
df.to_csv("data/simulated_data.csv", index=False) 

In [44]:
from util import (get_tensors, get_mask, init_weights, get_std_opt)
I = data['id'].nunique()

        ## split train/test
random_id = range(I) #np.random.permutation(range(I))
train_id = random_id[0:int(0.7*I)]
test_id = random_id[int(0.7*I):I]

train_data = data[data["id"].isin(train_id)]
test_data = data[data["id"].isin(test_id)]

print(train_data.shape)

batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(train_data)

(5329, 15)


/home/shijimao/Proj1/util.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"id_new"] = df.groupby(by="id").ngroup()
/home/shijimao/Proj1/util.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x_base[ii,jj,:] = torch.tensor(row.loc[list(base)])
/home/shijimao/Proj1/util.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x_long[ii,jj,:] = torch.tensor(row.loc[li

In [39]:
batch_long2, batch_base2, batch_mask2, batch_e2, batch_t2, obs_time2= get_tensors2(train_data)

In [41]:
sum(batch_long2 == batch_long)
sum(batch_base2 == batch_base)


tensor([[700, 700],
        [700, 700],
        [700, 700],
        [700, 700],
        [700, 700],
        [700, 700],
        [700, 700],
        [700, 700],
        [700, 700],
        [700, 700],
        [700, 700]])

In [ ]:
batch_size
for batch in range(0, len(train_id), batch_size):


In [27]:
sum(batch_mask)

tensor([700, 700, 650, 616, 576, 506, 448, 379, 309, 250, 195])

In [22]:
import numpy as np
np.max(train_data.loc[:,"visit"]) + 1


np.int64(11)

In [ ]:
import warnings 
warnings.filterwarnings("ignore")
n_epoch = 50
batch_size = 32
    
    
loss_values = []
loss1_list = []
loss2_list = []
for epoch in range(n_epoch):
    running_loss = 0
    train_id = np.random.permutation(train_id)
    for batch in range(0, len(train_id), batch_size):
        optimizer.zero_grad()
            
        indices = train_id[batch:batch+batch_size]
        batch_data = train_data[train_data["id"].isin(indices)]
            
        batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
        batch_long_inp = batch_long[:,:-1,:].to('cuda')
        batch_long_out = batch_long[:,1:,:].to('cuda')
        batch_base = batch_base[:,:-1,:].to('cuda')
        batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
        batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
        obs_time = obs_time.to('cuda')
        yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                        obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
        
        loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
        loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
        
        #loss = loss1 + loss2
        loss = multi_task_loss(loss1, loss2)
        
        loss.backward()
        scheduler.step()
        running_loss += loss
        loss1_list.append(loss1.tolist())
        loss2_list.append(loss2.tolist())
    loss_values.append(running_loss.tolist())
plt.plot((loss_values-np.min(loss_values))/(np.max(loss_values)-np.min(loss_values)), 'b-')


In [ ]:
def data_preprocessing(source):
    if source == 'JM':
        ##Load Data
        data = pd.read_pickle('data/simulated_data.pkl')
        I = data['id'].nunique()

        ## split train/test
        random_id = range(I) #np.random.permutation(range(I))
        train_id = random_id[0:int(0.7*I)]
        test_id = random_id[int(0.7*I):I]

        train_data = data[data["id"].isin(train_id)]
        test_data = data[data["id"].isin(test_id)]
    return train_id, train_data, test_id, test_data


def main(d_long = 3, d_base = 2, d_model = 32, nhead = 4, num_decoder_layers = 7,n_epoch = 50, batch_size = 32):
    train_id, train_data, test_id, test_data = data_preprocessing('JM')

    model = Transformer1(d_long=d_long, d_base=d_base, d_model=d_model, nhead=nhead,
                    num_decoder_layers=num_decoder_layers)
    model.to('cuda')
    model.apply(init_weights)
    model = model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
    scheduler = get_std_opt(optimizer, d_model=d_model, warmup_steps=200, factor=0.2)

    n_epoch = n_epoch 
    batch_size = batch_size 